In [ ]:
###python 多进程--实现多进程的几种方式

In [ ]:
# coding=utf-8

'''
方式一：os.fork()
pid=os.fork()
1.只在linux系统中生效，在win中无效
2.
'''

import os

pid=os.fork()

if pid==0:
    print("执行子进程，子进程pid={pid},父进程ppid={ppid}".format(pid=os.getpid(),ppid=os.getppid()))
else:
    print("执行子进程，子进程pid={pid},父进程ppid={ppid}".format(pid=pid,ppid=os.getpid()))

In [ ]:
# coding=utf-8

'''
方式二：multiprocessing模块，创建Process实例，传入执行函数作为参数
Process常用属性与方法：
    name:进程名
    pid：进程id
    run()，自定义子类时覆写
    start()，开启进程
    join(timeout=None)，阻塞进程
    terminate(),终止进程
    is_alive()，判断进程是否存活
'''

import os,time
from multiprocessing import Process

def worker():
    print("子进程执行，子进程pid={pid},父进程ppid={ppid}".format(pid=os.getpid(),ppid=os.getppid()))
    time.sleep(2)
    print("子进程结束>>>  子进程pid={0},父进程ppid={1}".format(os.getpid(),os.getppid()))
    
def main():
    print("主进程执行中>>> pid={0}".format(os.getpid()))
    
    processs=[]
    for i in range(2):
        p=Process(target=worker,name="worker"+str(i),args=())
        processs.append(p)
    
    for i in range(2):
        processs[i].start()
        
    for i in range(2):
        processs[i].join()
    
    print("主进程终止")

if __name__=='__main__':
    main()

In [ ]:
# coding=utf-8

'''
方式三：multiprocessing模块，派生Process子类，重写run方法
'''

import os,time
from multiprocessing import Process

class MyProcess(Process):
    def __init__(self):
        Process.__init__(self)
    def run(self):
        print("子进程执行，子进程pid={0},父进程ppid={1}".format(os.getpid(),os.getppid()))
        time.sleep(2)
        print("子进程结束>>>  子进程pid={0},父进程ppid={1}".format(os.getpid(),os.getppid()))

def main():
    print("主进程执行中>>> pid={0}".format(os.getpid()))  
    myp=MyProcess()
    myp.start()
    myp.join()
    print("主进程终止")

if __name__=='__main__':
    main()

In [ ]:
# coding=utf-8

'''
方式四：使用进程池pool
'''

import os,time
from multiprocessing import Pool

def worker(arg):
    print("子进程执行 子进程pid={},父进程ppid={},编号={}".format(os.getpid(),os.getppid(),arg))
    time.sleep(3)
    print("子进程结束 子进程pid={},父进程ppid={},编号={}".format(os.getpid(),os.getppid(),arg))

def main():
    print("主进程执行中 pid={0}".format(os.getpid()))  
    pools=Pool(5)
    for i in range(10):
        #pools.apply(worker,args=(i,))  #同步执行
        pools.apply_async(worker,args=(i,))  #异步执行
    
    # 关闭进程池，停止接受其它进程
    pools.close()
    # 阻塞进程
    pools.join()
    print("主进程终止")

if __name__=='__main__':
    main()

In [ ]:
# coding=utf-8

'''
生产者与消费者问题
'''
from atexit import register
from random import randrange
from multiprocessing import BoundedSemaphore,Lock,Process
from time import sleep,ctime

lock=Lock()
MAX=5
candytray=BoundedSemaphore(MAX)

def produce():
    lock.acquire()
    print('produce candy....')
    try:
        candytray.release()
    except ValueError:
        print('full, skipping')
    else:
        print('ok')
    lock.release()

def buy():
    lock.acquire()
    print('buy candy....')
    if candytray.acquire(False):
        print('OK')
    else:
        print('empty, skipping')
    lock.release()

def producer(loops):
    for i in range(loops):
        produce()
        sleep(randrange(3))

def consumer(loops):
    for i in range(loops):
        buy()
        sleep(randrange(3))

def main():
    print('starting at:',ctime())
    nloops=randrange(100,120)
    print('THE CANDY MACHINE (full with %d bars)!' % MAX)
    Process(target=consumer,args=(randrange(nloops,nloops+MAX+2),)).start()
    Process(target=producer,args=(nloops,)).start()

@register
def _atexit():
    print("all DONE at:",ctime())

if __name__=='__main__':
    main()